# Распознавание рукописных цифр из набора MNIST

Создадим сеть с использованием фреймворка PyTorch и обучим ее распознавать рукописные цифры. Это поможет вам понять как создается простая полносвязная сеть и как ее обучить.

## Imports

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms

## Загрузка данных MNIST и создание DataLoader

In [2]:
# Задаем параметры
batch_size = 256
learning_rate = 0.001
epochs = 10

# Загружаем данные MNIST и создаем DataLoader
transform = transforms.Compose(
                [transforms.ToTensor(), 
                transforms.Normalize((0.1307,), (0.3081,))
                ])
dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)



100%|██████████| 9912422/9912422 [00:06<00:00, 1558105.44it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 1590709.29it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:01<00:00, 1518957.26it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 6850244.07it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [3]:
# Определение простой полносвязной сети
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.flatten = nn.Flatten()
        # Преобразование входного изображения в одномерный тензор. 
        # Применяет операцию Flatten к входу x, которая преобразует многомерный тензор в одномерный. 
        # В данном случае, это используется для преобразования входных изображений размером 28x28 пикселей в одномерный тензор.
        self.fc1 = nn.Linear(28*28, 128)
        # Первый полносвязный слой с 128 нейронами
        self.relu = nn.ReLU()
        # Функция активации ReLU
        self.fc2 = nn.Linear(128, 10)
        # Второй полносвязный слой с 10 нейронами для классификации цифр от 0 до 9

    def forward(self, x):
        x = self.flatten(x)
        # Применение Flatten к входу
        x = self.fc1(x)
        # Проход через первый полносвязный слой
        x = self.relu(x)
        # Применение ReLU
        x = self.fc2(x)
        # Проход через второй полносвязный слой
        return x

In [4]:
# Инициализация модели, функции потерь и оптимизатора
model = SimpleNet()
# Создание экземпляра модели
criterion = nn.CrossEntropyLoss()
# Кросс-энтропийная функция потерь для задачи классификации
optimizer = optim.Adam(model.parameters(), lr=0.001)
# Оптимизатор Adam для обновления параметров модели

In [5]:
# Обучение модели
for epoch in range(10):
    # Цикл по эпохам
    for batch_idx, (data, target) in enumerate(train_loader):
        # Цикл по мини-пакетам внутри каждой эпохи
        optimizer.zero_grad()
        # Обнуление градиентов
        output = model(data)
        # Прямой проход: получение предсказаний модели
        loss = criterion(output, target)
        # Вычисление функции потерь
        loss.backward()
        # Обратное распространение: вычисление градиентов
        optimizer.step()
        # Обновление параметров модели по градиентам

        if batch_idx % 100 == 0:
            print(f'Epoch {epoch + 1}/{10}, Batch {batch_idx}/{len(train_loader)}, Loss: {loss.item():.4f}')
            # Вывод информации о процессе обучения

print('Training finished.')



Epoch 1/10, Batch 0/188, Loss: 2.3421
Epoch 1/10, Batch 100/188, Loss: 0.3988
Epoch 2/10, Batch 0/188, Loss: 0.2269
Epoch 2/10, Batch 100/188, Loss: 0.2010
Epoch 3/10, Batch 0/188, Loss: 0.1013
Epoch 3/10, Batch 100/188, Loss: 0.1495
Epoch 4/10, Batch 0/188, Loss: 0.0828
Epoch 4/10, Batch 100/188, Loss: 0.0755
Epoch 5/10, Batch 0/188, Loss: 0.1323
Epoch 5/10, Batch 100/188, Loss: 0.0563
Epoch 6/10, Batch 0/188, Loss: 0.0841
Epoch 6/10, Batch 100/188, Loss: 0.0777
Epoch 7/10, Batch 0/188, Loss: 0.0488
Epoch 7/10, Batch 100/188, Loss: 0.0626
Epoch 8/10, Batch 0/188, Loss: 0.0341
Epoch 8/10, Batch 100/188, Loss: 0.0962
Epoch 9/10, Batch 0/188, Loss: 0.0446
Epoch 9/10, Batch 100/188, Loss: 0.0440
Epoch 10/10, Batch 0/188, Loss: 0.0368
Epoch 10/10, Batch 100/188, Loss: 0.0602
Training finished.


In [7]:
# Оценка модели на тестовых данных
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data, target in test_loader:
        output = model(data)
        _, predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

accuracy = correct / total
print(f'Test Accuracy: {accuracy*100:2.2f}%')

Test Accuracy: 97.55%
